# **Project 2:**

In [75]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from src.Ward import Ward, initialize_wards
from src.Patient import Patient, initialize_patients
from src.simulation import simulation_loop, run_simulations,compute_gradient

## initialization

In [76]:
F_bed_capacity = 34 ##### Number to optimize #####

#patient type from 'A' to 'F'
patient_types = ['A', 'B', 'C', 'D', 'E', 'F']
bed_capacity = np.array([55,40,30,20,20, F_bed_capacity])
arrivals_pr_day = np.array([14.5,11.0,8.0,6.5,5.0, 13.0])
mean_length_of_stay = np.array([2.9,4.0,4.5,1.4,3.9, 2.2])
urgency_points = np.array([7,5,2,10,5, 0])

#reloaction probability
relocation_probability = np.array([
    [0.0, 0.05, 0.10, 0.05, 0.80, 0.00],
    [0.2, 0, 0.50, 0.15, 0.15, 0.00],
    [0.30, 0.20, 0, 0.20, 0.30, 0.00],
    [0.35, 0.30, 0.05, 0, 0.3, 0.00],
    [0.20, 0.10, 0.60 ,0.10, 0, 0.00],
    [0.20, 0.20, 0.20, 0.20, 0.20 ,0]
    ])

#Dataframe containing all the data (indexed by patient type)
df = pd.DataFrame({
    'Patient Type': patient_types,
    'Bed Capacity': bed_capacity,
    'Arrivals per day': arrivals_pr_day,
    'Mean Length of Stay': mean_length_of_stay,
    'Urgency Points': urgency_points
    }).set_index('Patient Type')

In [77]:
#sampling functions. Takes type as input and returns a sample with the corresponding distribution
arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda type: np.random.exponential(df["Mean Length of Stay"][type])
total_time = 31 #simulate 1 month

# Optimize Bed configuration

Create a new ward (F
∗
) in the system and allocate a minimal number of the current bed
resources to the new ward. Ensure that at least 95% of the type F
∗ patients are hospitalized in
Ward F
∗
. Use the ”urgency points” from Table 1 to balance the solution (prioritize wards that
need beds more than other wards).

In [141]:
#set seed!
np.random.seed(69)
F_occupied_prob = 1.0
penalties_F = []
F_occupied_probabilities = []
df.loc['F', 'Bed Capacity'] = 0
wards = initialize_wards(df) #list of wards
while F_occupied_prob > 0.05:
    #set F bed capacity in the dataframe
    performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 10, verbose = False)
    F_ward = wards[-1]
    F_occupied_prob = performance[F_ward]["Occupied probability"]
    print("F probability of rejection",F_occupied_prob, " with bed capacity ", wards[-1].capacity)
    penalties_F.append(performance["Weighted penalty"])
    F_occupied_probabilities.append(F_occupied_prob)
    
    
    gradients = compute_gradient(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 1, verbose = False)
    #find least negative gradient
    min_gradient = np.argmax(gradients[:-1])
    wards[min_gradient].capacity -=1
    wards[-1].capacity += 1
    for ward in wards:
        ward.reset_metrics()
wards[-1].capacity -=1
wards[min_gradient].capacity += 1
F_bed_capacity = len(penalties_F)

F probability of rejection 0.9999999999999999  with bed capacity  0
F probability of rejection 0.9663723145538172  with bed capacity  1
F probability of rejection 0.9371740066828078  with bed capacity  2
F probability of rejection 0.8978806691916963  with bed capacity  3
F probability of rejection 0.8653345671438247  with bed capacity  4
F probability of rejection 0.8311750999242015  with bed capacity  5
F probability of rejection 0.8019503789461226  with bed capacity  6
F probability of rejection 0.761697631322295  with bed capacity  7
F probability of rejection 0.725605739683655  with bed capacity  8
F probability of rejection 0.7117388137494347  with bed capacity  9
F probability of rejection 0.6732910798558855  with bed capacity  10
F probability of rejection 0.6335712250658336  with bed capacity  11
F probability of rejection 0.6087747315226727  with bed capacity  12
F probability of rejection 0.5770535297868049  with bed capacity  13
F probability of rejection 0.5373969596513113 

In [143]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(0, F_bed_capacity), y = penalties_F, name='Weighted penalty'))
fig.update_layout(title = "Penalty and rejection rate as a function of F bed capacity",
                  xaxis_title = "F bed capacity",
                  yaxis_title = "Weighted penalty")
fig.add_trace(go.Scatter(x = np.arange(0, F_bed_capacity), y = F_occupied_probabilities, yaxis = 'y2', name='Rejection probability'))
fig.update_layout(yaxis2 = dict(title = "Rejection probability", overlaying = 'y', side = 'right',
                                ))# Find the index where F_occupied_probabilities is closest to 0.05
index = (np.abs(np.array(F_occupied_probabilities) - 0.05)).argmin()

# Add a point at that index
fig.add_trace(go.Scatter(x=[np.arange(0, F_bed_capacity)[index-1]], 
                         y=[F_occupied_probabilities[index-1]], 
                         mode='markers', 
                         marker=dict(size=10, color='red'), 
                         name='5% Rejection'))
fig.show()
#save fig in plots
fig.write_image("plots/penalty_rejection.png")

In [150]:
#update the dataframe based on the optimal bed configuration
for i, ward in enumerate(wards):
    df.loc[ward.type, 'Bed Capacity'] = ward.capacity

#export dataframe to latex
df.to_latex("tables/ward_data.tex", float_format="%.1f")
#new bed configuration
df

,Bed Capacity,Arrivals per day,Mean Length of Stay,Urgency Points
Patient Type,,,,
A,49,14.5,2.9,7
B,28,11.0,4.0,5
C,22,8.0,4.5,2
D,16,6.5,1.4,10
E,16,5.0,3.9,5
F,34,13.0,2.2,0


In [78]:
#run the simulation with the optimal bed configuration
performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 10, verbose = False)
performance

{A Ward with 55 beds and 7 urgency points.: {'Occupied probability': 0.05016392858143146,
  'Estimated admissions': 482.9,
  'Estimated rejections': 26.0,
  'Estimated relocations': 11.6},
 B Ward with 40 beds and 5 urgency points.: {'Occupied probability': 0.23688668471289803,
  'Estimated admissions': 286.99999999999994,
  'Estimated rejections': 91.30000000000001,
  'Estimated relocations': 58.10000000000001},
 C Ward with 30 beds and 2 urgency points.: {'Occupied probability': 0.36013483791812967,
  'Estimated admissions': 211.0,
  'Estimated rejections': 120.8,
  'Estimated relocations': 66.39999999999999},
 D Ward with 20 beds and 10 urgency points.: {'Occupied probability': 0.053366940658573635,
  'Estimated admissions': 237.79999999999998,
  'Estimated rejections': 13.7,
  'Estimated relocations': 6.4},
 E Ward with 20 beds and 5 urgency points.: {'Occupied probability': 0.34572918556733206,
  'Estimated admissions': 146.9,
  'Estimated rejections': 78.80000000000001,
  'Estima

In [148]:
#convert performance to dataframe
df_performance = pd.DataFrame(performance).T
#exclude last row
df_performance = df_performance.iloc[:-1]
#change index
df_performance.index = [ward.type for ward in wards]
df_performance["Estimated lost"] = df_performance["Estimated rejections"] - df_performance["Estimated relocations"]
df_performance

,Occupied probability,Estimated admissions,Estimated rejections,Estimated relocations,Estimated lost
A,0.178110,458.9,100.6,30.4,70.2
B,0.459389,215.8,184.9,78.2,106.7
C,0.604787,150.7,232.8,90.9,141.9
D,0.222279,207.8,60.1,24.2,35.9
E,0.543174,136.7,163.8,38.1,125.7
F,0.047556,385.9,19.5,11.5,8.0


In [82]:
#create a stacked bar chart of the occupancy of the wards
import plotly.express as px
fig = px.bar(df_performance[["Estimated admissions", "Estimated relocations", "Estimated lost"]])
fig.update_layout(barmode='stack',
                    title='Occupancy of the wards after 1 month',
                    xaxis_title='Ward',
                    yaxis_title='Number of patients')
fig.show()
fig.write_image("plots/occupancy.png")
#save fig in plots

In [79]:
occupancy_last_sim = []
rejections_last_sim = []

for ward in wards:
    ward_occupancy = np.array(ward.occupancy)
    ward_rejections = np.array(ward.rejections)
    occupancy_last_sim.append(ward_occupancy[ward_occupancy[:,0] < total_time, :])
    rejections_last_sim.append(ward_rejections[ward_rejections < total_time])

In [80]:
fig = go.Figure()
for i,ward in enumerate(wards):
    occupancy = occupancy_last_sim[i]
    color = 'rgba(' + str(np.random.randint(0,256)) + ',' + str(np.random.randint(0,256)) + ',' + str(np.random.randint(0,256)) + ',1)'
    fig.add_trace(go.Scatter(x = occupancy[:,0], y = occupancy[:,1], mode = 'lines', name = ward.type, line = dict(color = color)))
    #plot upper bound as line for each ward
    fig.add_trace(go.Scatter(x = occupancy[:,0], y = ward.capacity*np.ones(occupancy.shape[0]), mode = 'lines', line = dict(dash = 'dash', color = color), name = ward.type + ' upper bound'))
    #plot rejections as crosses for each ward
    rejections = rejections_last_sim[i]
    fig.add_trace(go.Scatter(x = rejections, y = ward.capacity*np.ones(len(rejections)), mode = 'markers', marker = dict(symbol = 'x', size = 7, color = color, opacity=0.5), name = ward.type + ' rejections'))
fig.update_layout(title = 'Occupancy through time for each simulation in each ward', xaxis_title = 'Time[days]', yaxis_title = 'Occupancy', height = 600)
fig.show()
#save figure as png
fig.write_image("plots/occupancy_through_time.png")

 Create a new ward (F
∗
) in the system and allocate a minimal number of the current bed
resources to the new ward. Ensure that at least 95% of the type F
∗ patients are hospitalized in
Ward F
∗
. Use the ”urgency points” from Table 1 to balance the solution (prioritize wards that
need beds more than other wards).

In [142]:
performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 100, verbose = False)

[A Ward with 49 beds and 7 urgency points.,
 B Ward with 28 beds and 5 urgency points.,
 C Ward with 22 beds and 2 urgency points.,
 D Ward with 16 beds and 10 urgency points.,
 E Ward with 16 beds and 5 urgency points.,
 F Ward with 34 beds and 0 urgency points.]

In [135]:
#Number of beds required to ensure that 95% of the arrivals in F are admitted
F_bed_capacity_optimal = np.argmax(np.array(F_occupied_probabilities) < 0.05)
print("Number of beds required to ensure that 95% of the arrivals in F are admitted", F_bed_capacity_optimal)

Number of beds required to ensure that 95% of the arrivals in F are admitted 34


At some point increasing the number of beds in a ward will not dramatically decrease the penalty.

# Sensitivity analysis
Test the system’s sensitivity to the length-of-stay distribution by replacing the exponential
distribution with the log-normal distribution. Test the new distribution by gradually increasing
the variance (e.g. σ
2
i = 2/µ2
i
, 3/µ2
i
and 4/µ2
i
)

In [86]:
#sampling functions. Takes type as input and returns a sample with the corresponding distribution
#Test the system’s sensitivity to the length-of-stay distribution by replacing the exponential
#distribution with the log-normal distribution. Test the new distribution by gradually increasing
#the variance 

arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda var_scale : lambda type: np.random.lognormal(mean = np.log(df["Mean Length of Stay"][type]), sigma = np.sqrt(var_scale/df["Mean Length of Stay"][type]**2))

In [87]:
var_scales = np.linspace(1, 10, 10)
penalties = []
df.loc['F', 'Bed Capacity'] = F_bed_capacity_optimal
for var_scale in var_scales:
    wards = initialize_wards(df) #list of wards
    occupancy_time_function_var = occupancy_time_function(var_scale)
    performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function_var, n_simulations = 10, verbose = False)
    penalties.append(performance["Weighted penalty"])

In [88]:
#plot the results with PyPlot
fig = go.Figure()
fig.add_trace(go.Scatter(x = var_scales, y = penalties, name='Weighted penalty'))
fig.update_layout(title = "Weighted penalty as variance increases",
                  xaxis_title = "μ^2 * σ^2",
                  yaxis_title = "Weighted penalty")
fig.show()
#save fig in plots
fig.write_image("plots/penalty_variance.png")

Test the system’s sensitivity to the distribution of beds in the hospital.

In [89]:
#gradient of the weighted penalty as a function of the number of beds in each ward
arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda type: np.random.exponential(df["Mean Length of Stay"][type])

In [90]:
# Get the gradients for each ward
wards = initialize_wards(df) #list of wards
gradients = compute_gradient(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 100, verbose = False)

# Create a color array, setting the color to 'red' if the gradient is negative, 'blue' otherwise
colors = ['red' if gradient < 0 else 'blue' for gradient in gradients]

fig = go.Figure()
fig.add_trace(go.Bar(x=[ward.type for ward in wards], y=gradients, marker_color=colors))
fig.update_layout(title="Gradient of the weighted penalty with respect to the number of beds in each ward",
                  xaxis_title="Ward",
                  yaxis_title="Weighted penalty")
fig.show()
#save fig in plots
fig.write_image("plots/sensitivity.png")

valuate the result of increasing the total amount of beds in the system to for instance 170 or
180 beds. Also, what would be the result of having fewer beds

In [ ]:
#